#### Importação das bibliotecas

In [ ]:
import os
from dotenv import load_dotenv

import pandas as pd
import psycopg2 as pg
import sqlalchemy
from sqlalchemy import create_engine
import panel as pn
pn.extension()


#### Variáveis do ambiente

In [ ]:
# carrega as variáveis de ambiente do arquivo .env
load_dotenv()

# ler as variáveis de ambiente que tem as credênciais
DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")

# cria conexão com o Postgre usando o psycopg2 usando as variáveis de ambiente
# usado para executar comandos SQL
con = pg.connect(host = DB_HOST, dbname = DB_NAME, user = DB_USER, password = DB_PASS)

#### SQLAlchemy

In [ ]:
# define a string de conexão para o SQLAlchemy as variáveis de ambiente
cnx = f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}'

# cria o objeto engine do SQLAlchemy que será usado para conectar e executar comandos no banco
sqlalchemy.create_engine(cnx)


#### CRUD - Inventor


In [ ]:
# consulta incial para testar a conexão

query = "SELECT * FROM instituicao;"
df = pd.read_sql_query(query, cnx)

In [ ]:
# Inicializa o painel de visualização
pn.extension()
pn.extension('tabulator')
pn.extension(notifications = True)

In [ ]:
# ignorar campos vazios
flag = ''

In [ ]:
# campos para entrada de dados

# nome
nome_oficial = pn.widgets.TextInput(
    name = 'Nome Oficial', 
    value = '', 
    placeholder = 'Digite o nome', 
    disabled = False )

# email
email = pn.widgets.TextInput(
    name = 'Email',
    value = ' ',
    placeholder = "Digite um email",
    disabled = False
)

# cnpj
cnpj = pn.widgets.TextInput(
    name = 'CNPJ',
    value = ' ',
    placeholder = "Digite o CNPJ",
    disabled = False
)

# cep
cep = pn.widgets.TextInput(
    name = 'CEP',
    value = ' ',
    placeholder = "Digite o CEP",
    disabled = False
)

# estado
estados_brasileiros = [
    'AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 
    'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 
    'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO'
]

estado = pn.widgets.AutocompleteInput(
    name = 'Estado',
    options = estados_brasileiros,
    value = ' ',
    placeholder = "Digite o estado (Ex: CE)",
    case_sensitive = False,
    search_strategy = 'includes'
)

# cidade
cidade = pn.widgets.TextInput(
    name = 'Cidade',
    value = ' ',
    placeholder = "Digite a cidade",
    disabled = False
)

# bairro
bairro = pn.widgets.TextInput(
    name = 'Bairro',
    value = ' ',
    placeholder = 'Digite o bairro',
    disabled = False
)

# rua 
rua = pn.widgets.TextInput(
    name = 'Rua',
    value = ' ',
    placeholder = "Digite a rua",
    disabled = False
)

# numero
numero = pn.widgets.TextInput(
    name = 'Número',
    value = ' ',
    placeholder = "Digite o número",
    disabled = False
)

# tipo de instituição
tipo_instituicao = pn.widgets.RadioBoxGroup(
name = 'Tipo de Instituição',
options = ['Empresa', 'Universidade'],
disabled = False
)

# campo Empresa
setor_atividade = pn.widgets.TextInput(
    name = 'Setor de Atividade',
    value = ' ',
    placeholder = "Digite o setor de atividade",
    disabled = False
)

# campo Universidade
credenciamento_mec = pn.widgets.IntInput(
    name = 'Credenciamento e-MEC',
    value = 0,
    start = 0,
    step = 1,
    placeholder = "",
    disabled = False
)

# ID da instituição (para exclusão e atualização)
id_instituicao = pn.widgets.IntInput(
    name = 'ID da Instituição',
    value = 0,
    start = 0,
    step = 1,
    disabled = False
)

# criação dos botões
buttonConsultar = pn.widgets.Button(name='Consultar', button_type='default')
buttonInserir = pn.widgets.Button(name='Inserir', button_type='default')
buttonExcluir = pn.widgets.Button(name='Excluir', button_type='default')
buttonAtualizar = pn.widgets.Button(name='Atualizar', button_type='default')


##### Consultas

In [ ]:
# função para consultas todas as instituições
def queryAll():
    query = f"select * from instituicao;"
    df = pd.read_sql_query(query, cnx)
    return pn.widgets.Tabulator(df)

# função para consulta específica
def on_consultar():
    try:  
        query = f"select * from instituicao where ('{estado.value_input}'='{flag}' or cpf='{estado.value_input}')"
        df = pd.read_sql_query(query, cnx)
        table = pn.widgets.Tabulator(df)
        return table
    except:
        return pn.pane.Alert('Não foi possível realizar a consultar')
    

##### Inserção

In [ ]:
# função para inserção
def on_inserir():
    try:
        cursor = con.cursor()

        # verifica o tipo de instituição
        if not tipo_instituicao.value:
            return pn.state.notifications.error('Selecione o tipo de instituição!', 
                                                alert_type = 'warning')
        
        # insere na tabela instituicao
        cursor.execute("""
            INSERT INTO instituicao (nome_oficial, email, cnpj, cidade, bairro, numero, rua, estado, cep)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            RETURNING id_instituicao
          """, (
            nome_oficial.value_input,
            email.value,
            cnpj.value,
            cidade.value,
            bairro.value,
            numero.value,
            rua.value,
            estado.value,
            cep.value
          ))
        
        id_instituicao = cursor.fetchone()[0]
        
        # insere na tabela específica
        if tipo_instituicao.value == 'Empresa':
            if not setor_atividade.value_input:
                cursor.execute("ROLLBACK")
                return pn.state.notifications.error('Preencha o setor de atividade!', 
                                                    alert_type = 'warning')
            cursor.execute("""
                INSERT INTO empresa (id_instituicao, setor_atividade)
                VALUES (%s, %s)
            """, (
                id_instituicao,
                setor_atividade.value_input
            ))
        
        elif tipo_instituicao.value == 'Universidade':
            if credenciamento_mec.value_input == 0:
                cursor.execute("ROLLBACK")
                return pn.state.notifications.error('Preencha o credenciamento e-MEC!', 
                                                    alert_type = 'warning')
            cursor.execute("""
                INSERT INTO universidade (id_instituicao, credenciamento_mec)
                VALUES (%s, %s)
            """, (
                id_instituicao,
                credenciamento_mec.value_input
            ))
        
        con.commit()
        cursor.close()
        pn.state.notifications.success('Instituição inserida com sucesso!')
        return queryAll()
    
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        pn.state.notifications.error(f'Erro ao inserir instituição: {e}')

##### Exclusão

In [ ]:
# função para exclusão - utiliza o id da instituição
def on_excluir():
    try:
        cursor = con.cursor()

        # validação do ID
        if not id_instituicao.value:
            cursor.close()
            return pn.state.notifications.error('Preencha o ID da instituição!', 
                                                alert_type = 'warning')
        
        # verifica a existência do ID
        cursor.execute(
            "SELECT 1 FROM instituicao WHERE id_instituicao = %s",
            (id_instituicao.value,)
        )

        if not cursor.fetchone():
            cursor.close()
            return pn.state.notifications.error('ID não encontrado!', 
                                                alert_type = 'warning')
        
        # remover dependências nas tabelas
        cursor.execute("DELETE FROM empresa WHERE id_instituicao = %s", (id_instituicao.value,))
        cursor.execute("DELETE FROM universidade WHERE id_instituicao = %s", (id_instituicao.value,))
    
        # remover instituicao
        cursor.execute("DELETE FROM instituicao WHERE id_instituicao = %s", (id_instituicao.value,))

        con.commit()
        cursor.close()
        pn.state.notifications.success('Instituição excluída com sucesso!')
        return queryAll()
    
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        pn.state.notifications.error(f'Erro ao excluir instituição: {e}')

##### Atualização

In [ ]:
# função para atualização
def on_atualizar():
    try:
        cursor = con.cursor()

        # validação do ID
        if not id_instituicao.value:
            cursor.close()
            return pn.state.notifications.error('Preencha o ID da instituição!', 
                                                alert_type = 'warning')
        
        # verifica a existência do ID
        cursor.execute(
            "SELECT 1 FROM instituicao WHERE id_instituicao = %s",
            (id_instituicao.value,)
        )

        if not cursor.fetchone():
            cursor.close()
            return pn.state.notifications.error('ID não encontrado!', 
                                                alert_type = 'warning')
        
        # atualizar dados na tabela instituicao
        cursor.execute("""
            UPDATE instituicao
            SET nome_oficial = %s,
                email = %s,
                cnpj = %s,
                cidade = %s,
                bairro = %s,
                numero = %s,
                rua = %s,
                estado = %s,
                cep = %s
            WHERE id_instituicao = %s
          """, (
            nome_oficial.value_input,
            email.value,
            cnpj.value,
            cidade.value,
            bairro.value,
            numero.value,
            rua.value,
            estado.value,
            cep.value,
            id_instituicao.value
          ))
        
        con.commit()
        cursor.close()
        pn.state.notifications.success('Instituição atualizada com sucesso!')
        return queryAll()
    
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        pn.state.notifications.error(f'Erro ao atualizar instituição: {e}')

#### Chamadas

In [ ]:
# chama a função apropriada dependendo do botão clicado.
def table_creator(cons, ins, atu, exc):

    if cons:
        return on_consultar()
    if ins:
        return on_inserir()
    if atu:
        return on_atualizar()
    if exc:
        return on_excluir()

interactive_table = pn.bind(table_creator,
                             buttonConsultar.param.clicks,
                             buttonInserir.param.clicks,
                             buttonExcluir.param.clicks,
                             buttonAtualizar.param.clicks
                             )

#### Montagem do Layout

In [ ]:
pn.Row(
    pn.Column(
        '# Instituição CRUD',
        '---',

        '### Dados Básicos',
        nome_oficial,
        cnpj,
        email,

        '---',
        '### Endereço',
        rua,
        numero,
        bairro,
        cidade,
        estado,
        cep,

        '---',
        '### Tipo da Instituição',
        tipo_instituicao,
        setor_atividade,
        credenciamento_mec,

        '---',
        '### Ações',
        pn.Row(buttonConsultar, buttonInserir),
        pn.Row(buttonAtualizar, buttonExcluir),

        width=420
    ),

    pn.Column(
        '### Registros',
        interactive_table,
        width=1000
    )
).servable()
